In [3]:
import warnings
warnings.filterwarnings("ignore")

# Load dataset
The **Emotion** dataset is a collection of English social media posts labeled with one of six basic emotions: **anger, fear, joy, love, sadness,** and **surprise**. It is commonly used for training and evaluating emotion classification models in natural language processing tasks.


In [5]:
from datasets import load_dataset, DatasetDict

dataset = load_dataset("emotion", trust_remote_code=True)

README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [7]:
label_list = dataset['train'].features['label'].names
print(label_list)

['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']


In [8]:
model_name = "bert-base-uncased"
num_labels = len(label_list)

# Data preprocessing
Use BertTokenizer to tokenize the text

In [9]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [10]:
def preprocess(example):
  return tokenizer(example['text'], truncation=True, padding=True, return_tensors="pt")

tokenized_dataset = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

# Set up Training arguments and Trainer

We use a unified training arguments, compute metrics, and datacollator throughout this task for all following models.

In [11]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    logging_dir="./logs",
    label_names=['labels'],
    metric_for_best_model="eval_f1",
    report_to="tensorboard",
    load_best_model_at_end=True,
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

2025-06-01 05:44:06.799493: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748756647.023157      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748756647.087592      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [12]:
from evaluate import load
import numpy as np

metric = load("accuracy")
f1_metric = load("f1")
precision_metric = load("precision")
recall_metric = load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    return {
        "accuracy": metric.compute(predictions=predictions, references=labels)["accuracy"],
        "f1": f1_metric.compute(predictions=predictions, references=labels, average="weighted")["f1"],
        "precision": precision_metric.compute(predictions=predictions, references=labels, average="weighted")["precision"],
        "recall": recall_metric.compute(predictions=predictions, references=labels, average="weighted")["recall"],
    }


In [13]:
from transformers import EarlyStoppingCallback

early_stopping = EarlyStoppingCallback(early_stopping_patience=3)

# Full fine-tuning

In [14]:
from peft import get_peft_model, TaskType
from transformers import BertForSequenceClassification

In [15]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
import numpy as np

def count_trainable_parameters(model):
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    all_params = sum(p.numel() for p in model.parameters())
    trainable_percentage = 100 * trainable_params / all_params

    print(f"trainable params: {trainable_params:,} || all params: {all_params:,} || trainable%: {trainable_percentage:.4f}")

count_trainable_parameters(model)

trainable params: 109,486,854 || all params: 109,486,854 || trainable%: 100.0000


In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.436800,0.186824,0.926500,0.927241,0.931555,0.926500
2,0.138800,0.168374,0.937000,0.935590,0.938484,0.937000
3,0.117300,0.153666,0.939500,0.939747,0.940408,0.939500
4,0.096400,0.148560,0.938000,0.938323,0.940497,0.938000
5,0.076500,0.215513,0.938500,0.938287,0.938839,0.938500
6,0.052600,0.244514,0.936000,0.935299,0.936156,0.936000


TrainOutput(global_step=3000, training_loss=0.15306032498677571, metrics={'train_runtime': 1099.8996, 'train_samples_per_second': 727.339, 'train_steps_per_second': 22.729, 'total_flos': 4233000456579456.0, 'train_loss': 0.15306032498677571, 'epoch': 6.0})

In [18]:
result = trainer.evaluate(eval_dataset=tokenized_dataset["test"])
for key, value in result.items():
    print(f"{key}: {value:.4f}")
model.save_pretrained("./seq-full-model");

eval_loss: 0.1579
eval_accuracy: 0.9260
eval_f1: 0.9264
eval_precision: 0.9276
eval_recall: 0.9260
eval_runtime: 6.2680
eval_samples_per_second: 319.0810
eval_steps_per_second: 10.0510
epoch: 6.0000


# Partial fine-tuning
Freeze base model, only train the classifier head

In [19]:
frozen_model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
for param in frozen_model.bert.parameters():
    param.requires_grad = False
count_trainable_parameters(frozen_model)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 4,614 || all params: 109,486,854 || trainable%: 0.0042


In [20]:
frozen_trainer = Trainer(
    model=frozen_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

frozen_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.587800,1.564520,0.361500,0.223369,0.232320,0.361500
2,1.556900,1.552792,0.393000,0.288381,0.246638,0.393000
3,1.548500,1.545106,0.408500,0.298390,0.261715,0.408500
4,1.537600,1.535379,0.430500,0.330076,0.268431,0.430500
5,1.529100,1.523861,0.428000,0.316144,0.276651,0.428000
6,1.524900,1.519795,0.444500,0.342093,0.278109,0.444500
7,1.516500,1.509469,0.447000,0.337540,0.281266,0.447000
8,1.511500,1.504462,0.454000,0.346199,0.282746,0.454000
9,1.506200,1.499002,0.454000,0.344405,0.284413,0.454000
10,1.500400,1.497913,0.455500,0.351194,0.285762,0.455500


TrainOutput(global_step=15000, training_loss=1.4886580403645833, metrics={'train_runtime': 2679.1026, 'train_samples_per_second': 298.607, 'train_steps_per_second': 9.331, 'total_flos': 2.1164064916098816e+16, 'train_loss': 1.4886580403645833, 'epoch': 30.0})

In [21]:
result = frozen_trainer.evaluate(eval_dataset=tokenized_dataset["test"])
for key, value in result.items():
    print(f"{key}: {value:.4f}")
frozen_model.save_pretrained("./seq-frozen-model");

eval_loss: 1.4105
eval_accuracy: 0.4775
eval_f1: 0.3716
eval_precision: 0.3722
eval_recall: 0.4775
eval_runtime: 6.1727
eval_samples_per_second: 324.0050
eval_steps_per_second: 10.2060
epoch: 30.0000


# PEFT-LoRA

In [22]:
from peft import get_peft_model, LoraConfig, TaskType

base_model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

lora_config =  LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["query", "key", "value"],
)

lora_model = get_peft_model(base_model, lora_config)
for param in lora_model.classifier.parameters():
    param.requires_grad = True

lora_model.print_trainable_parameters()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 451,596 || all params: 109,933,836 || trainable%: 0.4108


In [23]:
lora_trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

lora_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.390600,1.050932,0.619500,0.519372,0.461206,0.619500
2,0.927200,0.792540,0.694500,0.640537,0.621333,0.694500
3,0.723200,0.585040,0.773000,0.735847,0.788410,0.773000
4,0.559100,0.440495,0.854500,0.847541,0.856815,0.854500
5,0.443400,0.334152,0.885500,0.883569,0.884845,0.885500
6,0.361800,0.289214,0.910000,0.909167,0.909327,0.910000
7,0.308200,0.253922,0.917500,0.917784,0.918760,0.917500
8,0.276700,0.232615,0.923500,0.924251,0.926196,0.923500
9,0.242500,0.216598,0.925500,0.925530,0.926302,0.925500
10,0.222700,0.208558,0.925500,0.926519,0.929911,0.925500


TrainOutput(global_step=9500, training_loss=0.3662460166529605, metrics={'train_runtime': 2858.5324, 'train_samples_per_second': 279.864, 'train_steps_per_second': 8.746, 'total_flos': 1.347206587841664e+16, 'train_loss': 0.3662460166529605, 'epoch': 19.0})

In [24]:
result = lora_trainer.evaluate(eval_dataset=tokenized_dataset["test"])
for key, value in result.items():
    print(f"{key}: {value:.4f}")
lora_model.save_pretrained("./seq-lora-model");

eval_loss: 0.1794
eval_accuracy: 0.9220
eval_f1: 0.9232
eval_precision: 0.9271
eval_recall: 0.9220
eval_runtime: 7.1103
eval_samples_per_second: 281.2830
eval_steps_per_second: 8.8600
epoch: 19.0000


## Fine-tuning Strategies Comparison

| Strategy              | Trainable Params | Epochs | Train Runtime (s) | Eval Accuracy | Eval Precision | Eval Recall | Eval F1  | Eval Loss |
|-----------------------|------------------|--------|--------------------|---------------|----------------|-------------|----------|-----------|
| Full Fine-tuning      | 109,486,854      | 6      | 1099.90            | **0.9260**     | **0.9276**      | 0.9260      | **0.9264** | 0.1579    |
| Classifier Only       | 4,614            | 30     | 2679.10            | 0.4775        | 0.3722         | 0.4775      | 0.3716   | 1.4105    |
| LoRA                  | 451,596          | 19     | 2858.53            | 0.9220        | 0.9271         | **0.9220**   | 0.9232   | **0.1794** |

## Conclusion

- **Full fine-tuning** achieves the highest F1 score and overall accuracy, with all model parameters updated. It is the most performant option but also the most resource-intensive.
- **LoRA** achieves **nearly identical performance** to full fine-tuning while training only **0.41%** of the parameters. It offers the best trade-off between efficiency and effectiveness.
- **Classifier-only fine-tuning** fails to learn effectively, with poor accuracy and F1 despite a longer training time and more epochs. It is not suitable for this task.